In [1]:
# Indusries
# Job Categories
# Locations ==> Suburb (optional)
# Job Title ==> Do (10) categories (cleaning) ==> Count Vectorizer
# Work Type
# Dates <== missing
# Job Description

# =============================================
#### Dummies
# =============================================
# ==> Industries
# ==> Location
# ==> Job Title

# =============================================
#### CountVectorizer
# =============================================
# ==> Job Title ==> Experiences columns (Senior, Junior, Entry-level)


# Step 1. Do Linear Regression

In [2]:
####
# Step 0: graph salary distribution (neg, pos skew)
# ==> EDA:
#     Contract DS (140 - 200k)
#     Contract DA (45 - 60k)
#     Contract (25%) | Fulltime (75%)
#     Industries (20-30) ==> to (10-15)
#     Job Titles ==> (DS [Junior, Senior/Manager, just DS], 
#                     DA [Junior, Senior/Manager, just DA]
#                     DE [Junior, Senior/Manager, just DE])
#                 ==> to (12-15)
#     Categories (200) ==> to (20)

# Step 1: Regression Problems
# ==> issues: 
#     not good model
# ==> treat as classification problem


In [3]:
import pandas as pd
import numpy as np
import re

# Own config
import config

# Folder
folder = './training/'
dest_folder = folder + 'Step5/'

target_file =  folder  + 'training_May-16-2019.csv'
dest_file =  dest_folder + 'clean__training_16052019_blank.csv'

In [4]:
frame = pd.read_csv(target_file, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Queensland Audit Office Investing in analytics...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Analyst - Audit Analytics Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/38867796?type=prom...,NaN
1,Our client has a significant track record of d...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Data Scientist - Analytics and AI Job in Brisb...,Full Time,https://www.seek.com.au/job/38999685?type=stan...,NaN
2,Working for this market leader within the Fina...,Information & Communication Technology,"Brisbane, CBD & Inner Suburbs",Head of Data Analytics & Performance Job in Br...,Full Time,https://www.seek.com.au/job/38986499?type=stan...,NaN
3,Full-Time | Permanent | Brisbane CBD About RAC...,Sales,"Brisbane, CBD & Inner Suburbs",Senior Data Analytics Consultant Job in Brisba...,Full Time,https://www.seek.com.au/job/38985472?type=stan...,NaN
4,Queensland Audit Office Investing in analytics...,Science & Technology,"Brisbane, CBD & Inner Suburbs",Analyst - Audit Analytics Job in Brisbane - SEEK,Full Time,https://www.seek.com.au/job/39008519?type=stan...,NaN


In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5529 entries, 0 to 5528
Data columns (total 7 columns):
content       5518 non-null object
industries    5518 non-null object
location      5518 non-null object
title         5518 non-null object
type_empl     5518 non-null object
url           5529 non-null object
wage          2088 non-null object
dtypes: object(7)
memory usage: 302.4+ KB


In [6]:
frame['wage'].unique()

array([nan, '$150-$175k + Super',
       'Great salary benefits + Laptop provided', ...,
       '$120000 - $135000 per annum', 'Great Salary available',
       '$50k - $60k p.a.'], dtype=object)

In [7]:
frame = frame[frame['title'].notnull()]
frame = frame[frame['wage'].isnull()]
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3430 entries, 0 to 5528
Data columns (total 7 columns):
content       3430 non-null object
industries    3430 non-null object
location      3430 non-null object
title         3430 non-null object
type_empl     3430 non-null object
url           3430 non-null object
wage          0 non-null object
dtypes: object(7)
memory usage: 214.4+ KB


In [8]:
frame['title'].value_counts()

Business Analyst Job in Melbourne - SEEK                                                        29
Business Analyst Job in Sydney - SEEK                                                           28
Data Analyst Job in Sydney - SEEK                                                               22
Business Analyst Job in Brisbane - SEEK                                                         17
Analyst - Audit Analytics Job in Brisbane - SEEK                                                16
Data Analyst Job in Melbourne - SEEK                                                            16
Senior Business Analyst Job in Sydney - SEEK                                                    14
Senior Business Analyst Job in Brisbane - SEEK                                                  13
Data Scientist Job in Melbourne - SEEK                                                          11
Technical Business Analyst Job in Sydney - SEEK                                                 10
Audit Seni

In [9]:
# Split Suburb out of content
def get_main_location(regex, sample):
#     print("In:", sample)
    result = re.search(regex, str(sample))

    if result:
#         print(result[0])
        result = result[0]
        result = result.replace(",","").lstrip().rstrip()
        return result
    elif len(str(sample)) > 3:
        return str(sample)
    return ''

# 
def get_suburb(regex, sample):
    result = re.sub(regex,"",str(sample))
    
    return result

In [10]:
frame['suburb'] = frame['location'].map(lambda row: get_suburb(r"(^[A-Za-z]+, )",row))
frame['location'] = frame['location'].map(lambda row: get_main_location(r"(^[A-Za-z]+, )",row))
frame['title'] = frame['title'].map(lambda row: row[:(row.find("Job")-1)] if row is not None else row)

In [11]:
frame['title'].value_counts()

Business Analyst                                                         74
Data Analyst                                                             43
Senior Business Analyst                                                  32
Data Scientist                                                           17
Analyst - Audit Analytics                                                16
Technical Business Analyst                                               14
Product Manager                                                          13
Finance Analyst                                                          12
Senior Data Analyst                                                      12
Marketing Coordinator                                                    12
Commercial Analyst                                                       12
Audit Senior - Performance Audit                                         10
Management Accountant                                                    10
Financial An

In [12]:
# Store all "keysearch" values in separate dataFrame.
# frame1 = []
# for value in fields:
#     result = frame[frame['title'] == value]
#     frame1.append(result)
# df = pd.concat(frame1)
# out_index = df.index.values
# df = df.reset_index(drop=True)
# df.head()
# print(df.shape)
# print("==============\n")
# frame.drop(out_index,inplace=True)
# frame.info()

In [13]:
# Round2: title include "keysearch" plus
# New dataFrame columns store the rest.
# titles = frame['title'].values
# new_titles = [''] * len(titles)
# title_left_over = [''] * len(titles)

# for i in fields:
#     for key, title in enumerate(titles):
#         if i in title:
#             # Push left over text to list
#             left_over = title.replace(i,"")
#             left_over = re.sub(r"([^a-zA-z\s])"," ",left_over)
#             left_over = re.sub(r"\s+"," ",left_over)

#             title_left_over[key] = left_over.lstrip().rstrip()
            
#             # Granted title as expected field
#             new_titles[key] = i
#         else:
#             title_left_over[key] = ''    
#             new_titles[key] = title
# df_title_lover = pd.DataFrame(title_left_over)
# frame['title'] = new_titles
# frame['left_over_title'] = title_left_over

In [14]:
titles = frame['title'].values
new_titles = frame['title'].values
experiences = [''] * len(titles)
title_left_over = [''] * len(titles)

for key, value in enumerate(new_titles):
    print("value: ", value)
    is_not_simple = True
    # Simple mapping level
    for exp in config.EXPERIENCE_LEVEL:
        if exp in value:       
            print("Exp in Exp_levels : ", exp,' ------- ' ,value)
            experiences[key] = exp
            title_left_over[key] = value
            new_titles[key] = value.replace(exp,"").rstrip().lstrip()            
            is_not_simple = False
            print("New title: ", new_titles[key])
            break
    
    # Dictionary mapping level
    print(config.TITLE_MAPPING_DICTIONARY)
    title_mapping_dictionary = config.TITLE_MAPPING_DICTIONARY
    if is_not_simple:
        for k_dict, v_dict in title_mapping_dictionary.items():
            for v_dict_item in v_dict:
                if v_dict_item in value:
                    print("Exp mapping dict : ", k_dict,' ------- ' ,value)
                    experiences[key] = k_dict
                    title_left_over[key] = value
                    new_titles[key] = value.replace(v_dict_item,"").rstrip().lstrip()
                    new_titles[key] = new_titles[key].replace(k_dict,"").rstrip().lstrip()
                    print("New title: ", new_titles[key])
                    break
                else:
                    experiences[key] = 'General'
    
    # Context mapping level as dictionary
    # For updating title only, NOT experiences
    context_mapping_dictionary = config.CONTEXT_MAPPING_DICTIONARY
    for k_dict, v_dict in context_mapping_dictionary.items():
        for v_dict_item in v_dict:
            if v_dict_item in value:
                print("[Title] mapping dict : ", k_dict,' ------- ' ,value)
                title_left_over[key] = value
#                 title_left_over[key] = value.replace(v_dict_item,".__.")
                new_titles[key] = k_dict
                break
        
    print("=================\n")
frame['title'] = new_titles
frame['experiences'] = experiences
frame['title_left_over'] = title_left_over

value:  Analyst - Audit Analytics
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Analyst - Audit Analytics

value:  Data Scientist - Analytics and AI
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Data Scientist - Analytics and AI
[Title] mapping dict :  Data Scientist  -------  Data Scientist - Analytics and AI

value:  Head of Data Analytics & Performance
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  Head of Data Analytics & Pe

value:  Data Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Data Analyst

value:  Coordinator Telecommunications Delivery
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Coordinator Telecommunications Delivery
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Coordinator Telecommunications Delivery
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

val

Exp in Exp_levels :  Senior  -------  Senior Manager, Student Recruitment (Australia)
New title:  Manager, Student Recruitment (Australia)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Marketing Assistant (Japanese/Vietnamese speaker)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Customer Success Manager
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  Customer Success Manager
New title:  Customer Success

value:  Power BI Reporting Specialist
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', '

{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Power BI Developer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Engineers  -------  Power BI Developer

value:  Senior Property Acquisitions Analyst
Exp in Exp_levels :  Senior  -------  Senior Property Acquisitions Analyst
New title:  Property Acquisitions Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Senior Property Acquisitions Analyst

value:  Senior PowerBI Developer
Exp in Exp_levels :  Senior  -------  Senior PowerBI

Exp in Exp_levels :  Senior  -------  Senior Integration Test Analyst
New title:  Integration Test Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Senior Integration Test Analyst

value:  Senior Scientist - Variant Curator 
Exp in Exp_levels :  Senior  -------  Senior Scientist - Variant Curator 
New title:  Scientist - Variant Curator
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Senior Scientist - Variant Curator 
Exp in Exp_levels :  Senior  -------  Senior Scientist - Variant Curator 
New title:  Scientist - Variant Curator
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'Genera

...

[Title] mapping dict :  Academic  -------  Research Executive

value:  Inventory Planner/Analyst
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Inventory Planner/Analyst

value:  Senior Management Accountant (12 month fixed-term contract)
Exp in Exp_levels :  Senior  -------  Senior Management Accountant (12 month fixed-term contract)
New title:  Management Accountant (12 month fixed-term contract)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Account  -------  Senior Management Accountant (12 month fixed-term contract)

value:  EKA CTRM System Support Analyst - Sydney
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', '

[Title] mapping dict :  Data Analyst  -------  Business Analyst-Banking 

value:  Tax Analyst (Permanent or Contract)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Tax Analyst (Permanent or Contract)

value:  Tax Analyst (Permanent or Contract)
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Tax Analyst (Permanent or Contract)

value:  Product Manager
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
Exp mapping dict :  Senior  -------  Product Manager
New title:  Produ


value:  Administrative Assistant
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Duty Trader
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Senior Software Business Analyst |National Organisation - Software/Product House
Exp in Exp_levels :  Senior  -------  Senior Software Business Analyst |National Organisation - Software/Product House
New title:  Software Business Analyst |National Organisation - Software/Product House
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Senior Software Business An

[Title] mapping dict :  Data Scientist  -------  Senior Data Scientist

value:  Data Scientist - Machine Learning // C++ // python
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Data Scientist - Machine Learning // C++ // python
[Title] mapping dict :  Data Scientist  -------  Data Scientist - Machine Learning // C++ // python

value:  Data Scientist - Machine Learning // C++ // python
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Data Analyst  -------  Data Scientist - Machine Learning // C++ // python
[Title] mapping dict :  Data Scientist  -------  Data Scientist - Machine Learning // C++ // python

value:  Platform Data Engine

Exp in Exp_levels :  Senior  -------  Senior Android Developer
New title:  Android Developer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Software Developer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}
[Title] mapping dict :  Engineers  -------  Software Developer

value:  Junior Full Stack Engineer
Exp in Exp_levels :  Junior  -------  Junior Full Stack Engineer
New title:  Full Stack Engineer
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'Snr', 'Sr', 'sr', 'Supervisor', 'Master'], 'General': ['all', 'All', 'Experienced'], 'Junior': ['Jr', 'Junior', 'jr', 'Tableau']}

value:  Case Manager - Full time Permanent
{'Senior': ['Senior', 'Lead', 'Head', 'Manager', 'Director', 'S

In [15]:
# frame['experiences'].value_counts()

In [16]:
frame['title'].value_counts()

Data Analyst                                                                    1094
Data Scientist                                                                   173
Account                                                                          169
Engineers                                                                        160
Finance                                                                          127
Academic                                                                          70
Marketing Coordinator                                                             12
Audit  - Performance Audit                                                        10
Clasoom Teacher                                                                    9
Developer                                                                          9
Marketing                                                                          8
BI Consultant                                                    

In [17]:
frame['title_left_over'].value_counts()

                                                                         970
Business Analyst                                                          74
Data Analyst                                                              43
Senior Business Analyst                                                   32
Data Scientist                                                            17
Analyst - Audit Analytics                                                 16
Technical Business Analyst                                                14
Product Manager                                                           13
Senior Data Analyst                                                       12
Commercial Analyst                                                        12
Finance Analyst                                                           12
Financial Analyst                                                         10
Audit Senior - Performance Audit                                          10

In [18]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3430 entries, 0 to 5528
Data columns (total 10 columns):
content            3430 non-null object
industries         3430 non-null object
location           3430 non-null object
title              3430 non-null object
type_empl          3430 non-null object
url                3430 non-null object
wage               0 non-null object
suburb             3430 non-null object
experiences        3430 non-null object
title_left_over    3430 non-null object
dtypes: object(10)
memory usage: 294.8+ KB


In [19]:
# Turn type_empl to 2 sub-categories
wtype = frame['type_empl'].values
new_wtype = [''] * len(wtype)

for key, value in enumerate(wtype):
    if value in 'Full Time' or value in 'Part Time':
        new_wtype[key] = 'Permanent'
    else:
        new_wtype[key] = 'Contract'
frame['type_empl'] = new_wtype

In [20]:
frame.head()

,content,industries,location,title,type_empl,url,wage,suburb,experiences,title_left_over
0,Queensland Audit Office Investing in analytics...,Information & Communication Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38867796?type=prom...,NaN,CBD & Inner Suburbs,General,Analyst - Audit Analytics
1,Our client has a significant track record of d...,Information & Communication Technology,Brisbane,Data Scientist,Permanent,https://www.seek.com.au/job/38999685?type=stan...,NaN,CBD & Inner Suburbs,General,Data Scientist - Analytics and AI
2,Working for this market leader within the Fina...,Information & Communication Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38986499?type=stan...,NaN,CBD & Inner Suburbs,General,Head of Data Analytics & Performance
3,Full-Time | Permanent | Brisbane CBD About RAC...,Sales,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38985472?type=stan...,NaN,CBD & Inner Suburbs,Senior,Senior Data Analytics Consultant
4,Queensland Audit Office Investing in analytics...,Science & Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/39008519?type=stan...,NaN,CBD & Inner Suburbs,General,Analyst - Audit Analytics


# OUTPUT

In [21]:
output_df = pd.DataFrame(columns =frame.columns)

for key, value in enumerate(config.FIELDS):
    print(value)
    df_temp = frame[frame['title'] == value]
    
    print(df_temp.shape)
    output_df = pd.concat([output_df,df_temp])

Data Analyst
(1094, 10)
Data Scientist
(173, 10)


In [22]:
output_df = output_df.reset_index(drop=True)
output_df.head()

,content,industries,location,title,type_empl,url,wage,suburb,experiences,title_left_over
0,Queensland Audit Office Investing in analytics...,Information & Communication Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38867796?type=prom...,NaN,CBD & Inner Suburbs,General,Analyst - Audit Analytics
1,Working for this market leader within the Fina...,Information & Communication Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38986499?type=stan...,NaN,CBD & Inner Suburbs,General,Head of Data Analytics & Performance
2,Full-Time | Permanent | Brisbane CBD About RAC...,Sales,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/38985472?type=stan...,NaN,CBD & Inner Suburbs,Senior,Senior Data Analytics Consultant
3,Queensland Audit Office Investing in analytics...,Science & Technology,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/39008519?type=stan...,NaN,CBD & Inner Suburbs,General,Analyst - Audit Analytics
4,Queensland Audit Office Investing in analytics...,Government & Defence,Brisbane,Data Analyst,Permanent,https://www.seek.com.au/job/39017806?type=stan...,NaN,CBD & Inner Suburbs,General,Analyst - Audit Analytics


In [23]:
output_df.to_csv(dest_file,sep='|',index=False)